In [1]:
import json
import re
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.cluster import AgglomerativeClustering
from time import gmtime, strftime
from datetime import datetime


In [2]:
#Reading file provide SQL
def read_csv(name_df,path):
    name_df = pd.read_csv(path+".csv",index_col=0)
    return name_df
#df = pd.read_csv("data-1579021640655.csv") 

#delete column or columns from a data frame
#   --argument:
#    --dataframe name 
#    --columns names
def delete_column_df(df_name,*args):
    for name_col in args:
        df_name = df_name.drop(columns=[name_col])
    return df_name        

#Transform dataframe to array_np to calculate idf
def df_to_array_np(name_df):
    name_array=name_df.to_numpy().astype(np.float)
    return name_array
#exclude repeated attributes from the list
def exclude_repeated_att(list_att):
    update_list = list(dict.fromkeys(list_att))
    return update_list


In [3]:
df= read_csv('df','data')
#df = df.head(45720)#-----------------
df_original = df
df_original.sort_values('visit_id', inplace=True, ascending=True ,kind='mergesort')
df_a = df
array_id2 =df_a['visit_id'].values

In [4]:
#-----------------------Pre-processing - Cada procedimento será um atributo na tabela  --------# 
arr = []
arr_t=[]
count=0
for i, row in df.iterrows():
    obj = json.loads(row['internacao_json'])
    string_concept = ''
    arr_t.append(obj['visit_concept_name'])
    string_concept+=obj['visit_concept_name']+'&'
    #id = str(obj['visit_occurrence_id'])
    if(obj['procedimentos'] != None):
        for procedure_name in obj['procedimentos']:
            nome_do_procedimento = str(procedure_name['procedure_ocurrence_concept_name'])
            string_concept += nome_do_procedimento + "&"
            arr_t.append(nome_do_procedimento)
    else:
        count+=1
    arr.append(string_concept)


In [5]:
#----------------------------------Pegar id + nome das ocorrencias(AUX no Mapeamento Final)-------------#
count=0
linhas =0
sera =0
laco_for=0
#--------LEMBRAR de concatenar --------#string_conc_oco= ""
#df_id_name_ocorrence = pd.DataFrame(['ID_visit','Nome_Procedimento'],[0,0])
df_id_name_ocorrence = pd.DataFrame()
lista_concep_name_ocorrencia=[]
id_visit=[]
cod_ocorrencia =[]
for i, row in df.iterrows():
    linhas+=1
    
    obj = json.loads(row['internacao_json'])
   
    if(obj['ocorrencias'] != None):
        laco_for +=1
        for procedure_name in obj['ocorrencias']:
            nome_do_procedimento = str(procedure_name['condition_ocurrence_concept_name'])
            lista_concep_name_ocorrencia.append(nome_do_procedimento)
            codigo_oco= str(procedure_name['condition_concept_id'])
            cod_ocorrencia.append(codigo_oco)
            id_visit.append(row['visit_id'])         
        
    else:
        lista_concep_name_ocorrencia.append("NULO")
        cod_ocorrencia.append("00000000000")
        id_visit.append(row['visit_id'] )
        
        count+=1
#----------------------------------Criando para o MAPEAMENTO Final----------#
list_of_tuples = list(zip(id_visit, cod_ocorrencia,lista_concep_name_ocorrencia))
df_id_nome_concept = pd.DataFrame(list_of_tuples, columns = ['ID_visita','Cod_Ocorrencia' ,'Nome_Ocorrencia']) 


#-------------------Pre-processing- Manter apenas Letra e numero------------# 
#input = arr_t
#arr_t = [x.lower() for x in input] 

#input1 = arr
#arr = [x.lower() for x in input1] 

#arr_t = exclude_repeated_att(arr_t)
#arr_t
#print(len(arr_t))

#arr = [re.sub(' +', ' ', elem) for elem in arr]
#arr = [re.sub("[^A-Za-z\d\&]", "_", elem) for elem in arr]





#---------------------- Pre-processing---- para lidar apenas como uma unica palavra, trocando sepador para o metodo IDFID___
arr = list(map(lambda s: s.replace('&' , ' '), arr))
#print(len(arr))


In [18]:
#---- Normalizando e criando um novo dataframe ---- Com VALORES  TF-IDF
tfidf = TfidfVectorizer()
x = tfidf.fit_transform(arr)
df_tfidf = pd.DataFrame(x.toarray(), columns=tfidf.get_feature_names())
print(df_tfidf)

df_tfidf.to_csv("representation_BOARN.csv",index = False)


      125   19   1o   24   67  67ga   72   ab  abdomem  abdomen  ...  \
0     0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0      0.0      0.0  ...   
1     0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0      0.0      0.0  ...   
2     0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0      0.0      0.0  ...   
3     0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0      0.0      0.0  ...   
4     0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0      0.0      0.0  ...   
...   ...  ...  ...  ...  ...   ...  ...  ...      ...      ...  ...   
5527  0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0      0.0      0.0  ...   
5528  0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0      0.0      0.0  ...   
5529  0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0      0.0      0.0  ...   
5530  0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0      0.0      0.0  ...   
5531  0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0      0.0      0.0  ...   

          vias  vii  viiic     visit  vitais  vitamina   vp  western   xi  xii  
0     0.000000  0.0    0.0  0.285849     0.0       0.0

In [12]:
type(arr)

list

In [26]:
df_text_bow= pd.DataFrame(arr)

In [32]:
df_text_bow.columns = ['records']

In [34]:
df_text_bow.to_csv("records_bow.csv",index = False)
